In [1]:
# The code was removed by Watson Studio for sharing.

# Monitoring your predictive model - Configuration

## Introduction


Watson Studio tracks and measures outcomes from your models. This notebook walks through configuring the monitoring system for the model we previously deployed with AutoAI. After configuration we will set up monitors to ensure the model remains fair, explainable and compliant while running. We will also set up a monitor to detect and help correct drift in accuracy while the model is in production. 

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

## Table of Contents
* [0. Setup](#0.-Setup)
    * [0.1. Manage Access & User Inputs](#0.1-Manage-Access-&-User-Inputs)
    * [0.2 Install additional packages and Load Dependencies](#0.2-Install-additional-packages-and-Load-Dependencies)
    * [0.3 Create OpenScale and Watson Machine Learning Client Connections](#0.3-Create-OpenScale-and-Watson-Machine-Learning-Client-Connections)
* [1. Create Model Subscription](#1.-Create-Model-Subscription)
    * [1.1 Create OpenScale Client & Binding](#1.1-Create-OpenScale-Client-&-Binding)
    * [1.2 Create OpenScale Subscription](#1.2-Create-OpenScale-Subscription)
* [2. Perform Preliminary Steps](#2.-Perform-preliminary-steps)
    * [2.1 Gather Training Statistics](#2.1-Gather-Training-Statistics)
    * [2.2 Run Drift Trainer](#2.2-Run-Drift-Trainer)
* [3. Payload Logging](#3.-Payload-Logging)
    * [3.1 Leverage automatic payload logging](#3.1-Leverage-automatic-payload-logging)
    * [3.2 Check the content of the payload logging table](#3.2-Check-the-content-of-the-payload-logging-table)
    
* [4. Configure Monitors](#4.-Configure-Monitors)
    * [4.1 Quality Monitoring](#4.1-Quality-Monitoring)
    * [4.2 Fairness Monitoring](#4.2-Fairness-Monitoring)
    * [4.3 Explainability Monitoring](#4.3-Explainability-Monitoring) 
    * [4.4. Drift](#4.4-Drift)


# 0. Setup

## 0.1 Manage Access & User Inputs

### 0.1.0 Insert project token and Cloud API key

### Insert project token
Before executing this notebook on IBM Cloud , you need to insert a project access token at the top of this notebook <br>
To do so click on **More -> Insert project token** in the top-right menu section and run the cell <br>

### Insert your Cloud API Key
This notebook requires an API key and instance location to be able to use WML deploy and score models.
To access your API key, go to cloud.ibm.com, then click Manage > Access(IAM)


In [2]:
import os

CLOUD_API_KEY = ''
location = os.environ['RUNTIME_ENV_REGION']


### 0.1.1 Enter Model Deployment Details & Define Thresholds

The `SPACE_NAME`, `MODEL_NAME`, and `DEPLOYMENT_NAME` are the names of the assets in the deployment space that you previously linked to this project and AutoAI model. The thresholds correspond to monitors we will configure throughout the notebook. 

- QUALITY_THRESHOLD - Sends an alert in the OpenScale UI when the quality of predictions falls below 50%
- FAIRNESS_THRESHOLD - Sends an alert when the bias falls below 95%
- DRIFT_THRESHOLD - Sends an alert when the predictions see a drop in performance of greater that 10%


<div class="alert alert-warning">
  <strong>Attention!</strong><br><br>The user MUST be: <br>
    1. an admin of the Watson Studio for Monitoring service (i.e. Watson OpenScale) <br>
    2. and also an admin of the Deployment Space in Watson Studio that hosts your model.   
</div>

In [3]:
# Enter the name of the Deployment space, together with the Model and Deployment name
SPACE_NAME = 'Accelerator Space'
MODEL_NAME = 'CODE_Customer_Attrition'
DEPLOYMENT_NAME = 'CODE_Customer_Attrition_Deployment'


# Define Thresholds 
QUALITY_THRESHOLD = .5 
QUALITY_MIN_RECORDS = 50
FAIRNESS_THRESHOLD = .95 
DRIFT_THRESHOLD = 0.1 
DRIFT_MIN_RECORDS = 50


### 0.1.2 Confirm OpenScale User Access

It's common practice for the system admin to restrict access to the OpenScale service to a few users. If you are unsure if you have access to the service. Check with your admin before running this notebook. Keep in mind if you do not have proper access, the client may allow you to connect and get part of the way through before breaking when configuring certain monitors. Once you have the correct credential information you need to add the user to the deployment space. 

From the hamburger menu on the top left corner of the screen under Analyze select Analytics Deployments. From that screen you should see the `CustomerAttrition` space, click on it and navigate to the Access Control tab. From here you can see a list of all the users with access. If the username with access to OpenScale is not there, add them by clicking add collaborators, entering in the username and changing the role to admin.

## 0.2 Install additional packages and Load Dependencies

We install `ibm_wos_utils`, which is package used to configure the drift monitor later in the notebook. 

In [4]:
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install ibm_wos_utils --no-cache -q

<br>

### Load Dependencies

In [5]:
# GENERAL PYTHON IMPORTS
import os
import time
import uuid
from tqdm import tqdm_notebook as tqdm
import json
import datetime

import pandas as pd
import numpy as np

# WML IMPORT: ACCESS DEPLOYED MODELS
import ibm_watson_machine_learning

# OPENSCALE IMPORTS
import ibm_watson_openscale
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

from ibm_watson_openscale.utils.training_stats import TrainingStats
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Asset, AssetDeploymentRequest, AssetPropertiesRequest, Target
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord


from ibm_wos_utils.drift.drift_trainer import DriftTrainer

<br>

## 0.3 Create OpenScale and Watson Machine Learning Client Connections

In this step we create the Watson Machine Learning and OpenScale clients, get the `deployment_space_id`, and set the default space.

In [6]:
## Watson Machine Learning
from ibm_watson_machine_learning import APIClient


WML_CREDENTIALS = {
                   "url": 'https://' + location + '.ml.cloud.ibm.com',
                   "apikey": CLOUD_API_KEY
}

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.232'

In [7]:
## Watson OpenScale Client

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment if using IAM token
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.20'

### 0.3.1 Get additional information from Watson Machine Learning

In this step, we identify the `space_id` corresponding to our space, and get additional information about our model and its deployment, using the names provided in 0.1.1.

In [8]:
try:
    space_id = ''
    for space in wml_client.spaces.get_details()['resources']:
        if space['entity']['name'] == SPACE_NAME:
            space_id = space['metadata']['id']
            print(SPACE_NAME)
            print(f"space_id: {space_id}\n")
            wml_client.set.default_space(space_id)
            break

    if space_id == '':
        print("Deployment space", SPACE_NAME, "not found.")
except:
    print("Deployment Space couldn't be found, please check again the spelling on the first cell (user input)")
    

try:     
    model_id = [x['metadata']['id'] for x in wml_client.repository.get_model_details()['resources'] if x['metadata']['name'] == MODEL_NAME][0]
    model_details = wml_client.repository.get_details(model_id)
    model_url = wml_client.repository.get_model_href(model_details)
    print(MODEL_NAME)
    print(f"model_id: {model_id}\n")
except: 
    print("Model name couldn't be found, please check again the spelling on the first cell (user input)")

try: 
    deployment_id = [x['metadata']['id'] for x in wml_client.deployments.get_details()['resources'] if x['entity']['name'] == DEPLOYMENT_NAME][0]
    deployment_details = wml_client.deployments.get_details(deployment_id)
    deployment_url = wml_client.deployments.get_scoring_href(deployment_details)
    print(DEPLOYMENT_NAME)
    print(f"deployment_id: {deployment_id}\n")
except: 
    print("Deployment Name couldn't be found, please check again the spelling on the first cell (user input)")



Accelerator Space
space_id: 423a676d-dd96-4a55-97d2-325cab64171c

CODE_Customer_Attrition
model_id: 24cd355e-9283-411c-ad27-eb5b4b90813c

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
CODE_Customer_Attrition_Deployment
deployment_id: cedbb13c-dfc3-4b6f-bbed-8e66f7f07c05



In [9]:
model_details = wml_client.repository.get_details(model_id)

input_data_schema = model_details['entity']['schemas']['input'][0]['fields']
feature_list = [x['name'] for x in input_data_schema]
categorical_features = [x['name'] for x in input_data_schema if x['type'] == 'string']
# label_column = wml_client.repository.get_details(model_id)['entity']['label_column']

label_column = "TARGET"

all_columns = feature_list.copy()
all_columns.append(label_column)

<br>  

# 1. Create Model Subscription

A subscription is the connection between OpenScale and your deployed model. It's important to make sure all the parameters you are inputting to your configuration are complete and accurate. Creating a subscription that is not configured correctly can lead to difficulty in when using the various monitors later on.

## 1.1 Create OpenScale Client & Binding

Before you can create the subscription to your deployed MODEL, you must first create the binding between OpenScale and the deployment SPACE in WML. Creating the binding will give OpenScale access to all the deployed assets within that space.

In [10]:
wos_client.data_marts.show()

None,None,False,active,2022-08-05 17:01:28.422000+00:00,6cfe5811-9736-4656-b699-6e8710710186


In [11]:
data_mart_details = wos_client.data_marts.list().result.data_marts
data_mart_id = wos_client.data_marts.list().result.to_dict()['data_marts'][0]['metadata']['id']
print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 6cfe5811-9736-4656-b699-6e8710710186


In [12]:
SERVICE_PROVIDER_NAME = "Customer Attrition"
SERVICE_PROVIDER_DESCRIPTION = "Monitoring Subscription for the Customer Attrition Model"

In [13]:
# OpenScale can have more than one service binding with the same name,
# set the flag below to True if you want to replace a service binding with the same name
replace_existing = True
service_provider_id = None
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    print(f"{service_instance_name:60} {service_provider.metadata.id:50}")
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        if replace_existing:
            wos_client.service_providers.delete(service_provider_id)
            print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))
            service_provider_id = None

Hiring preproduction space                                   df1da5bb-7d53-4e63-a042-0156622ce580              
Attrition preproduction space                                52c902d7-b363-4570-b289-1254817e6501              
Attrition production space                                   3413cdc1-b557-4e49-a8f2-2e31bbe79c59              


In [14]:
# Add a service binding if one with the target name doesn't already exist
from ibm_watson_openscale import *

if service_provider_id is None:
        try: 
            added_service_provider_result = wos_client.service_providers.add(
                    name=SERVICE_PROVIDER_NAME,
                    description=SERVICE_PROVIDER_DESCRIPTION,
                    service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
                    deployment_space_id=space_id,
                    operational_space_id="production",
                    credentials= WML_CREDENTIALS,
                    background_mode=False
                ).result
            service_provider_id = added_service_provider_result.metadata.id
        except Exception as e: 
            print(f"ERROR: {e}")
            print("Check if your USER, who is the admin for OpenScale, is also included in the deployment space as an Admin.")






 Waiting for end of adding service provider b3601664-4762-4f13-a377-1c2e6f1e9b7d 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [15]:
wos_client.service_providers.show(limit=10)

b7bc6824-e151-4762-b6c5-b2f287308b1b,active,Customer Attrition,watson_machine_learning,2022-08-09 15:33:45.718000+00:00,b3601664-4762-4f13-a377-1c2e6f1e9b7d
b7bc6824-e151-4762-b6c5-b2f287308b1b,active,Hiring preproduction space,watson_machine_learning,2022-08-05 17:05:39.603000+00:00,df1da5bb-7d53-4e63-a042-0156622ce580
b7bc6824-e151-4762-b6c5-b2f287308b1b,active,Attrition preproduction space,watson_machine_learning,2022-08-05 17:02:57.881000+00:00,52c902d7-b363-4570-b289-1254817e6501
b7bc6824-e151-4762-b6c5-b2f287308b1b,active,Attrition production space,watson_machine_learning,2022-08-05 17:01:53.839000+00:00,3413cdc1-b557-4e49-a8f2-2e31bbe79c59


<br>

## 1.2 Create OpenScale Subscription

We only want OpenScale to monitor the features our model is actually making predictions on. Below is the complete list of features as well as a subset of categorical features our model is using for prediction.

To create the subscription OpenScale needs the `model_uid`, `problem_type`, `label_column`, `feature_columns`, and `categorical_columns`. We gather the `model_uid` from our Watson Machine Learning client and pass in the rest of the information as parameters. This model is a binary classification problem that makes predictions for the variable `TARGET`.

In [16]:
subscription_details = wos_client.subscriptions.add(
            data_mart_id=data_mart_id,
            service_provider_id=service_provider_id,
            asset=Asset(
                asset_id=model_id,
                name=MODEL_NAME,
                url=model_url,
                asset_type=AssetTypes.MODEL,
                input_data_type=InputDataType.STRUCTURED,
                problem_type=ProblemType.BINARY_CLASSIFICATION
            ),
            deployment=AssetDeploymentRequest(
                deployment_id=deployment_id,
                name=DEPLOYMENT_NAME,
                deployment_type=DeploymentTypes.ONLINE,
                url=deployment_url
            ),
            asset_properties=AssetPropertiesRequest(
                label_column=label_column,
                probability_fields=['probability'],
                prediction_field='prediction',
                feature_fields=feature_list,
                categorical_fields=categorical_features,
            )
        ).result

subscription_id = subscription_details.metadata.id
print(subscription_id)

# will be used when we set up monitors, to point to the subscription we just created:
target_subscription = Target(target_type=TargetTypes.SUBSCRIPTION, target_id=subscription_id)

e1ae8546-9245-434a-9539-15c2e867aaf8


# 2. Perform preliminary steps

Before proceeding to the rest of the setup using the OpenScale setup, we perform some local steps, which outputs will be used in downstream steps.

## 2.1 Gather Training Statistics

In order to set up some of the OpenScale monitors (explainability and fairness) we will need training statistics describing our training data. In this step, we load the data used for training and compute these statistics, to be used in the following section.

To get training statistics, we could either point to training data stored in Db2 or Cloud Object Storage (not covered here, see [this section](https://dataplatform.cloud.ibm.com/docs/content/wsj/model/wos-training-data-schema.html) of the documentation, or, load the dataset as a pandas dataset and use the `TrainingStats` class. Therefore, we first load the training dataset, and then pass it to `TrainingStats` along with some parameters:

In [17]:
df_training = pd.read_csv(project.get_file('data_prep_output.csv'))
fairness_inputs = {
    "fairness_attributes": [
        {"feature": "CUSTOMER_GENDER_Male",
         "majority": [[0.5,1]],
         "minority": [[0, 0.49]],
         "threshold": FAIRNESS_THRESHOLD
         }
    ],
    "favourable_class": ['NO'],
    "unfavourable_class": ['YES'],
    "min_records": QUALITY_MIN_RECORDS
}
    
input_parameters = {
    "label_column": label_column,
    "feature_columns": feature_list,
    "categorical_columns": categorical_features,
    "fairness_inputs": fairness_inputs,  
    "problem_type" : 'binary'  
}


This cell will take a couple minutes to run:

In [18]:
try: 
    training_stats = TrainingStats(df_training, input_parameters, explain=True, fairness=True, drop_na=True)
    training_stats_json = training_stats.get_training_statistics()
    print("Training Stats calculated correctly")
    ## Save the training stats

    import json

    data_file = "training_stats.json"
    with open(data_file, 'w') as fp:
        json.dump(training_stats_json, fp)
except: 
    print("The function TrainingStats had an issue so we will leverage the training_stats.json file")
    my_file = project.get_file("training_stats.json")
    training_stats_json = json.load(my_file)

Training Stats calculated correctly


## 2.2 Run Drift Trainer

In order to monitor for Drift, OpenScale needs to train a Drift Model that learns the data used for training along with the predictions. It's important to perform this step before setting up payload logging: when we train the Drift model, we will be making some scoring requests to the deployment and we want to avoid logging those, as they are not "real" scoring requests coming from end users. Therefore we make sure to perform this step before the next section (payload logging setup).

In [19]:
def score(data, deployment_id=deployment_id):
    """
    From the generate_drift_detection_model docs:
    A function that accepts a dataframe with features as columns and returns a tuple of numpy array
    of probabilities array of shape `(n_samples,n_classes)` and numpy array of prediction vector of shape `(n_samples,)`
    """
    scoring_payload = {'input_data': [{'values': data[feature_list]}]}
    scoring_result = wml_client.deployments.score(deployment_id, scoring_payload)
    
    probas = np.array([x[1] for x in scoring_result['predictions'][0]['values']])
    preds = np.array([x[0] for x in scoring_result['predictions'][0]['values']])
    return probas, preds

In [20]:
drift_success = False 

try: 

    drift_detection_input = {
        "feature_columns": feature_list,
        "categorical_columns": categorical_features,
        "label_column": label_column,
        "problem_type": 'binary'
    }

    drift_trainer = DriftTrainer(df_training, drift_detection_input)

    # Note: Two column constraints are not computed beyond two_column_learner_limit(default set to 200)
    # User can adjust the value depending on the requirement
    drift_trainer.generate_drift_detection_model(score, batch_size=df_training.shape[0])
    drift_trainer.learn_constraints(two_column_learner_limit=200)
    drift_trainer.create_archive()
    
    ## Once created the drift model, now let's upload it to our OpenScale Subscription
    with open("drift_detection_model.tar.gz", 'rb') as model:
        project.save_data("drift_detection_model.tar.gz", model, overwrite=True)
    
    upload_response = wos_client.monitor_instances.upload_drift_model(
        model_path="drift_detection_model.tar.gz",
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
    ).result
    
    drift_success = True
    print("Drift Model created and correctly uploaded in the subscription")

except Exception as e:
    print(f"ERROR: {e}\n")
    print("Unable to create the Drift Model - skip the drift monitoring set up for now")

Learning two feature constraints...: 100%|██████████| 658/658 [00:03<00:00, 197.95constraints/s]
Drift Model created and correctly uploaded in the subscription


# 3. Payload Logging

The way that OpenScale monitors activity is through payload logging. Once you score the model, the details of the record are sent to OpenScale, saved, and then can be tracked by the various monitors. Automatic payload logging exists between IBM Watson OpenScale and IBM Watson Machine Learning when they are provisioned in the same IBM Cloud account, or for IBM Watson OpenScale for IBM Cloud Pak for Data in the same cluster.

In this section, we leverage Automatic Payload Logging. 

### 3.1 Leverage automatic payload logging

In this case, all we need is to score some new data using the model deployment, and OpenScale will automatically watch that deployment and log the payloads for us. In other terms, if we were using this code from an application, we wouldn't need to add any OpenScale-related code or logic for the Payload Logging to be done.

In [21]:
payload_data_set_id = wos_client.data_sets.list(
        target_target_id=subscription_id,
        target_target_type="subscription",
        type="payload_logging"
    ).result.data_sets[0].metadata.id 

In [22]:
payload_data_set_id

'642df504-6ba5-43ac-88e7-cd7d805b1e1b'

In [23]:
time.sleep(20)

X_train = df_training.copy()

X_train = X_train.drop([label_column], axis=1).dropna()
feature_list = X_train.columns.tolist()
data = X_train.dropna().sample(n=55).values.tolist()

from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

## scoring request
payload = {"input_data":[{
        "fields": feature_list,
        "values": data
}],}
## scoring results from Watson Machine Learning
res = wml_client.deployments.score(deployment_id,payload)
print('SCORED')
print(len(data))


## logging the payload (request + results) in Watson OpeScale
response_data = res
request_data = payload
records = [PayloadRecord(request=request_data, response=response_data, response_time=10)]
wos_client.data_sets.store_records(data_set_id = payload_data_set_id, request_body=records)
print('\nLOGGED')

SCORED
55

LOGGED


### 3.2 Check the content of the payload logging table

Here we make sure that our payloads got logged. It takes a couple seconds in order for the payloads to be stored, therefore we wait before checking the content of the payload table:

In [24]:
import time
time.sleep(20)

pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print(f"Number of records in the payload logging table: {pl_records_count}")

Number of records in the payload logging table: 831


# 4. Configure Monitors

Now that the subscription has been made and payloads have been logged we can choose what monitors to set up. For our claims classifcation model, we want to monitor the following

- Quality 
- Fairness
- Explainability
- Drift


## 4.1 Quality Monitoring

For our model we will set the threshold quality to 50% where OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second parameter supplied, `min_records`, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

### 4.1.0 Enable Quality Monitor

In [25]:
parameters = {
    "min_feedback_data_size": QUALITY_MIN_RECORDS
}

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target_subscription,
    parameters=parameters
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id




 Waiting for end of monitor instance creation 1c950ae1-88c0-4ea7-8e54-d37d28a23de7 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### 4.1.1 Feedback Logging

Feedback data is essential to maintain an unbiased model. You must upload feedback data to the IBM Watson OpenScale service on a regular basis to ensure that your model takes into account up-to-date data that may indicate changes in the context of your predictive application. With a feedback loop, the system learns continuously by monitoring the effectiveness of predictions and retraining when needed. Monitoring and using the resulting feedback are at the core of machine learning. The following information is meant to help you with formatting and uploading your feedback data.

- CSV: File sizes are currently limited to 8MB. use the UI to upload the file
- JSON: Use this notebook to send the feedback in the Json format as follows. 

Quality needs a baseline dataset that includes the outcomes to be able to detect the change. The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [26]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                             target_target_id=subscription_id, 
                                             target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "9e962480-7b4e-4ec7-886d-57bd5b07f8df",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/181ed6cc388f47bd9d862fe066f9cfce:6cfe5811-9736-4656-b699-6e8710710186:data_set:9e962480-7b4e-4ec7-886d-57bd5b07f8df",
        "url": "/v2/data_sets/9e962480-7b4e-4ec7-886d-57bd5b07f8df",
        "created_at": "2022-08-09T15:35:53.197000Z",
        "created_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad",
        "modified_at": "2022-08-09T15:35:53.711000Z",
        "modified_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad"
      },
      "entity": {
        "data_mart_id": "6cfe5811-9736-4656-b699-6e8710710186",
        "name": "e1ae8546-9245-434a-9539-15c2e867aaf8_feedback",
        "description": "e1ae8546-9245-434a-9539-15c2e867aaf8_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "e1ae8546-9245-434a-9539-15c2e867aaf8"
        },
    

We now need to send a validation dataset as our feedback dataset. We want OpenScale to see and learn what a good classification looks like. Of course, you need to have the target variable included so that OpenScale can learn from the feedback. 

In [27]:
X_validation = df_training.copy() # For setting up purposes let's use the training data for now. 

feature_list = X_validation.columns.tolist()
data = X_validation.dropna().sample(n=55).values.tolist()

payload = {"input_data":[{
        "fields": feature_list,
        "values": data
}],}


feedback_data = payload['input_data']
wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)





 Waiting for end of storing records with request id: cbbdaeba-f7cc-4f08-b991-57c102ffbd7e 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




### 4.1.2 Run Quality Monitoring

In [28]:
time.sleep(20)
quality_run = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id,background_mode=True)
quality_run_id = quality_run.result.to_dict()['metadata']['id']

In [29]:
wos_client.monitor_instances.get_run_details(quality_monitor_instance_id, quality_run_id).result.to_dict()

{'metadata': {'id': 'db3f5c2f-a5fa-4a4a-aa87-fb3aed0e581b',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/feb0a088323a45db90b8dd694b314c53:6cfe5811-9736-4656-b699-6e8710710186:run:db3f5c2f-a5fa-4a4a-aa87-fb3aed0e581b',
  'url': '/v2/monitor_instances/1c950ae1-88c0-4ea7-8e54-d37d28a23de7/runs/db3f5c2f-a5fa-4a4a-aa87-fb3aed0e581b',
  'created_at': '2022-08-09T15:36:33.586000Z',
  'created_by': 'IBMid-0600029VTW'},
 'entity': {'parameters': {'min_feedback_data_size': 50,
   'total_records_processed': 55},
  'status': {'state': 'running',
   'queued_at': '2022-08-09T15:36:33.572000Z',
   'started_at': '2022-08-09T15:36:35.266000Z',
   'updated_at': '2022-08-09T15:36:35.339000Z',
   'operators': [{'id': 'original',
     'status': {'state': 'queued',
      'started_at': '2022-08-09T15:36:33.859000Z'}}]}}}

## 4.2 Fairness Monitoring


The code below configures fairness monitoring for our model. We will configure the monitor on to alert us for categorical features `Customer_Gender`.

In each case, we must specify:

- Which model feature to monitor
- One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
- One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
- The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 5 records have been added. 

### 4.2.0 Enable Fairness Monitor

In [30]:
parameters = {
    "features": fairness_inputs["fairness_attributes"],
    "favourable_class": fairness_inputs["favourable_class"],
    "unfavourable_class": fairness_inputs["unfavourable_class"],
    "min_records": fairness_inputs["min_records"]
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target_subscription,
    parameters=parameters
).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation ae8d549f-46cc-4cd9-848c-4ab964299709 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### 4.2.1 Run Fairness Monitor

In [31]:
time.sleep(40)
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=True)

In [32]:
run_id = run_details.result.to_dict()['metadata']['id']
run_id

'5c977d9c-ef60-4a5a-83cf-ee640e25d320'

In [33]:
wos_client.monitor_instances.get_run_details(fairness_monitor_instance_id, run_id).result.to_dict()

{'metadata': {'id': '5c977d9c-ef60-4a5a-83cf-ee640e25d320',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/feb0a088323a45db90b8dd694b314c53:6cfe5811-9736-4656-b699-6e8710710186:run:5c977d9c-ef60-4a5a-83cf-ee640e25d320',
  'url': '/v2/monitor_instances/ae8d549f-46cc-4cd9-848c-4ab964299709/runs/5c977d9c-ef60-4a5a-83cf-ee640e25d320',
  'created_at': '2022-08-09T15:37:34.721000Z',
  'created_by': 'IBMid-0600029VTW'},
 'entity': {'parameters': {'bias_end_time': '2022-08-09T15:37:02.158237Z',
   'class_label': 'prediction',
   'favourable_class': ['NO'],
   'features': [{'feature': 'CUSTOMER_GENDER_Male',
     'majority': [[0.5, 1]],
     'minority': [[0, 0.49]],
     'threshold': 0.8}],
   'group_bias_computed_at': '2022-08-09T15:36:57.316840Z',
   'last_processed_ts': '2022-08-09 15:36:52.693969',
   'last_run_id': '526a4695-c009-4925-b75b-70018974c9c4',
   'manual_labeling_table_name': 'Manual_Labeling_e1ae8546-9245-434a-9539-15c2e867aaf8',
   'min_records': 50,
   'next_evaluatio

Fairness takes time to log so we wait for the table to populate before viewing the results

In [34]:
time.sleep(10)
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2022-08-09 15:36:53.910210+00:00,fairness_value,62df3771-1071-42d1-9c13-1fba3f6a80cf,102.69999999999999,80.0,None,"['feature:CUSTOMER_GENDER_Male', 'fairness_metric_type:fairness', 'feature_value:0-0.49']",fairness,ae8d549f-46cc-4cd9-848c-4ab964299709,526a4695-c009-4925-b75b-70018974c9c4,subscription,e1ae8546-9245-434a-9539-15c2e867aaf8
2022-08-09 15:36:53.910210+00:00,fairness_value,d7d89604-16a8-4364-985d-127d92a43c1d,102.69999999999999,80.0,None,"['feature:CUSTOMER_GENDER_Male', 'fairness_metric_type:debiased_fairness', 'feature_value:0-0.49']",fairness,ae8d549f-46cc-4cd9-848c-4ab964299709,526a4695-c009-4925-b75b-70018974c9c4,subscription,e1ae8546-9245-434a-9539-15c2e867aaf8


## 4.3 Explainability Monitoring

Explainability provides us with detailed information on a per observation basis. This will allow the end-user to look at the features that are contributing to a particular prediction and take appropriate action.


### 4.3.0 Enable Explainability Monitor

In [35]:
parameters = {
    "enabled": True,
    "training_statistics": training_stats_json["explainability_configuration"]
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target_subscription,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 2b7ad680-567b-4b54-b55d-7004477f63cb 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### 4.3.1 Run Explainability Monitor

This step extracts the first `scoring_id` from the `payload_logging` table and runs the explainability monitor for that record. The output is a dictionary that contains all of the explanation weights and corresponding features.

In [36]:
explanation_types = ["lime", "contrastive"]

In [37]:
sample_payload = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [sample_payload["records"][0]["entity"]["values"]["scoring_id"]]
print(f"Running explanations on scoring IDs: {scoring_ids}")

expl_task_trigger = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(expl_task_trigger)

expl_id = expl_task_trigger.metadata.explanation_task_ids[0]
expl_task_result = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=expl_id).result
print(expl_task_result)

Running explanations on scoring IDs: ['fea3e510-e11f-4ab8-b3c7-f1f51e48535b-1']
{
  "metadata": {
    "explanation_task_ids": [
      "ac2e067a-5dc2-4786-be96-86a4660b564a"
    ],
    "created_by": "IBMid-0600029VTW",
    "created_at": "2022-08-09T15:38:20.527220Z"
  }
}
{
  "metadata": {
    "explanation_task_id": "ac2e067a-5dc2-4786-be96-86a4660b564a",
    "created_by": "IBMid-0600029VTW",
    "created_at": "2022-08-09T15:38:20.527220Z"
  },
  "entity": {
    "status": {
      "state": "in_progress"
    },
    "scoring_id": "fea3e510-e11f-4ab8-b3c7-f1f51e48535b-1"
  }
}


Explanation task might take some time so waiting for completion to see the results. 

In [38]:
time.sleep(60)
expl_task_result = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=expl_id).result
expl_task_result.to_dict()

{'metadata': {'explanation_task_id': 'ac2e067a-5dc2-4786-be96-86a4660b564a',
  'created_by': 'IBMid-0600029VTW',
  'created_at': '2022-08-09T15:38:20.527220Z',
  'updated_at': '2022-08-09T15:39:27.292606Z'},
 'entity': {'status': {'state': 'finished'},
  'asset': {'id': '24cd355e-9283-411c-ad27-eb5b4b90813c',
   'name': 'CODE_Customer_Attrition',
   'input_data_type': 'structured',
   'problem_type': 'binary',
   'deployment': {'id': 'cedbb13c-dfc3-4b6f-bbed-8e66f7f07c05',
    'name': 'CODE_Customer_Attrition_Deployment'}},
  'input_features': [{'name': 'NUM_ACCOUNTS_WITH_INVESTMENT_OBJECTIVE_SECURE_GROWTH',
    'value': '1.0',
    'feature_type': 'numerical'},
   {'name': 'CUSTOMER_SUMMARY_NUMBER_OF_ACCOUNTS',
    'value': '3.0',
    'feature_type': 'numerical'},
   {'name': 'CUSTOMER_FAMILY_SIZE',
    'value': '3.0',
    'feature_type': 'numerical'},
   {'name': 'CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT',
    'value': '80000.0',
    'feature_type': 'numerical'},
   {'name': 'CUSTOMER_

### 4.3.2 View Explaination Results

We recommend to use the OpenScale User Interface to see how Lime and Contrastive explanations results are displayed.  

## 4.4 Drift 

The drift test measures two types of changes.


1. **Drop in accuracy**<br>
structured binary and multi-class classification models only
Watson OpenScale estimates the drop in accuracy of the model at runtime. The model accuracy could drop if there is an increase in transactions similar to those which the model was unable to evaluate correctly in the training data.


2. **Drop in data consistency**<br>
Watson OpenScale estimates the drop in consistency of the data at runtime as compared to the characteristics of the data at training time.
A drop in model accuracy and data consistency may lead to a negative impact on the business outcomes associated with the model.

In [39]:
if drift_success: 

    targetD = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id

    )
    parametersD = {
        "min_samples": DRIFT_MIN_RECORDS,
        "drift_threshold": DRIFT_THRESHOLD,
        "train_drift_model": False
    }

    drift_monitor_details = wos_client.monitor_instances.create(
        data_mart_id=data_mart_id,
        background_mode=False,
        monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
        target=targetD,
        parameters=parametersD
    ).result

else: 
    print("Postponed the creation of the drift monitor due to the impossibility to calculate the Drift model as per above paragraph 2.2")




 Waiting for end of monitor instance creation 8f3308da-3867-4fb4-bff6-55e02729cde2 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## The setup of the monitor-subscription is now complete. 
Now, go to the UI and take a look at the other features available. 


------------